
## 목표: 한국어 신문기사를 가지고 n-gram에 기반한 Language Modeling 구현
- File Handling
- Preprocessing
- Word Count
- Efficient한 N-gram 구현 


## 내용:

- NLRW1900000011.json 은 강원일보의 뉴스기사이다.
- 파일에서 신문기사만을 추출하고, 각 기사를 문장단위로 분리한 후 문장 처음과 끝에 각각 < s, /s >표시를 붙여주기
- 뉴스기사를 가급적 정리하기. Regular expression, 한국어 형태소 분석기 사용, translation 등으로 필요없는 글자들을 제거
- 정리된 데이터를 가지고 실제 기사에 나오는 **'특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다'** 문장의 원래의 문장 카운트에 의한 확률, unigram, Bigram, Trigram 언어모델에 의한 확률 구하기

## Data Loading and Preprocessing


In [1]:
# Korean Sentence Splitter: https://pypi.org/project/kss/
!pip install kss

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251557 sha256=f667cb1b95e5f667180d88dd757566f062fa10486a5a53855d7ccb4104d40e3e
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/My Drive/Colab Notebooks/2020-PoscoICT/Data/NLRW1900000011.json'

In [4]:
import pandas as pd
import json, re, time
import kss
from collections import defaultdict

In [5]:
START_TOKEN = '<s '
END_TOKEN = ' /s>'
NUMBER_TOKEN = '<NUM>'
TARGET_SENT = START_TOKEN + '특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다' + END_TOKEN

In [6]:
letter_number_pattern = re.compile('[^가-힣a-zA-Z0-9]+')
number_pattern = re.compile('[0-9]+')
doublespace_pattern = re.compile('\s+')
def sentences_from_json(filepath):
    '''
    주어진 json 파일에서 신문기사만을 추출하고, 각 기사를 문장단위로 분리한 후 전처리한다
    Parameters
    ----------
    filepath : string
        json 파일 경로
    Returns
    ----------
    sentences : list of string
        1) kss 라이브러리를 통해 문장 tokenization
        2) 한글, 영어 및 숫자만 추출한 뒤 숫자는 <NUMBER>로 치환
    '''
    with open(filepath) as json_file:
        json_data = json.load(json_file)
        sentences = []
        for document in json_data['document']:
            for paragraph in document['paragraph']:
                sentences.extend([doublespace_pattern.sub(' ', number_pattern.sub(NUMBER_TOKEN, letter_number_pattern.sub(' ', sentence)) ).strip() for sentence in kss.split_sentences(paragraph['form'])])        
    json_file.close()
    sentences = ['<s '+ sent +' /s>' for sent in sentences if sent.strip() != '']
    return sentences

sentences = sentences_from_json(filepath=train_path)
print('Total number of sentences:', len(sentences))

Total number of sentences: 93424


In [7]:
# 전처리 완료된 문장 예시 출력
[sent for sent in sentences if NUMBER_TOKEN in sent][:5]

['<s 올해는 <NUM>년 동계올림픽 국내 후보도시가 결정되는 해다 /s>',
 '<s 도정은 평창의 <NUM>수 도전의 당위성과 이룰 수 있다는 근거를 도민들에게 다시 한 번 소상히 밝혀야 한다 /s>',
 '<s 그래야 강원도가 통일<NUM>번지로 우뚝 설 수 있다 /s>',
 '<s 도청 도교육청과 체육 핫 라인 을 구축해 월 <NUM>회 정기회의와 유기적인 협조 체제를 구축 기관별 역할을 분담해 강원체육의 기초부터 새롭게 다져 체육 꿈나무 육성에 주력할 것이다 /s>',
 '<s 이는 <NUM>시 군생활체육협의회와 도종목별연합회의 회장을 비롯한 임직원의 협조가 있었기에 가능했다 /s>']

## 문장 빈도에 의한 확률

- P(특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다) 

\begin{equation}
    P(< s 특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다 /s >) \approx \frac{count(특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다)}{count(모든 문장)}
\end{equation}


In [8]:
def sentence_probability_by_sentence_count(target_sent, sentences):
    '''
    모든 문장들에 대해 주어진 타겟 문장이 등장한 빈도수를 계산하여 '문장 빈도에 의한 확률'을 구한다
    Parameters
    ----------
    target_sent : string
        문장에 대한 확률을 측정할 타겟 문장
    sentences : list of string 
        전처리 완료된 모든 문장 list
    Returns
    ----------
    sent_prob : float
        문장 빈도에 의한 확률
    '''
    return len([sent for sent in sentences if sent == target_sent]) / len(sentences)

print('문장 빈도에 의한 확률: ', sentence_probability_by_sentence_count(TARGET_SENT, sentences))

문장 빈도에 의한 확률:  1.0703887651995204e-05


## Unigram Language Model에 의한 확률
### Unigram Model (k=1): $P(w_1 w_2 ... w_n) \approx \prod_{i} P(w_i)$

\begin{equation}
    P(x_1) \approx \frac{count(x_1)}{count(N)}
\end{equation}

- P(*< s 특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다 /s >*)


In [9]:
def to_ngrams(words, n):
    '''
    단어들로부터 ngram을 추출한다
    Parameters
    ----------
    words : list of string
        띄어쓰기를 기준으로 토크나이징 완료된 단어들
    n : int
        unigram일 경우 1, bigram일 경우 2, trigram일 경우 3, ...
    Returns
    ----------
    ngrams : list of tuples
        unigram일 경우 [(단어1,), (단어2), (단어3,), ...]
        bigram일 경우 [(단어1, 단어2), (단어2, 단어3), (단어3, 단어4), ...]
        trigram일 경우 [(단어1, 단어2, 단어3), (단어2, 단어3, 단어4), (단어3, 단어4, 단어5), ...]
    '''
    ngrams = []
    for b in range(0, len(words) - n + 1):
        ngrams.append(tuple(words[b:b+n]))
    return ngrams

def ngram_df_using_defaultdict(flattened_sentences, n):
    '''
    단어들로부터 ngram을 추출한다
    Parameters
    ----------
    flattened_sentences : string
        모든 문장들 사이에 빈칸을 두고 하나의 string으로 합친 것
    n : int
        unigram일 경우 1, bigram일 경우 2, trigram일 경우 3, ...
    Returns
    ----------
    df : pandas DataFrame
        모든 문장에 대해 추출한 ngram token 및 빈도수 (count) 값을 기록한 dataframe
    '''
    ngram_counter = defaultdict(int)
    tokens = to_ngrams(flattened_sentences.split(), n)
    for token in tokens:
        ngram_counter[token] += 1
    return pd.DataFrame(list(ngram_counter.items()), columns=['token', 'count'])

flattened_sentences = ' '.join(sentences)
unigram_df = ngram_df_using_defaultdict(flattened_sentences, n=1)

# unigram 빈도수를 기록한 dataframe 내 데이터 예시 출력
unigram_df.head()

,token,count
0,"(<s,)",93424
1,"(새로운,)",750
2,"(희망,)",75
3,"(공유하고,)",16
4,"(새,)",229


In [10]:
def sentence_prob_by_ngram_language_model(target_tokens, numerator_df, denominator_df=None):
    '''
    ngram language model을 사용할 때 주어진 문장에 대한 확률을 구한다
    Parameters
    ----------
    target_tokens : list of tuples
        unigram일 경우 [(단어1,), (단어2), (단어3,), ...]
        bigram일 경우 [(단어1, 단어2), (단어2, 단어3), (단어3, 단어4), ...]
        trigram일 경우 [(단어1, 단어2, 단어3), (단어2, 단어3, 단어4), (단어3, 단어4, 단어5), ...]
    numerator_df : Maximum Likelihood Estimation 식에서 분자를 계산할 때 사용할, token 및 count 정보가 담긴 pandas DataFrame
        unigram일 경우 unigram_df, bigram일 경우 bigram_df, trigram일 경우 trigram_df, ...
    denominator_df : Maximum Likelihood Estimation 식에서 분모를 계산할 때 사용할, token 및 count 정보가 담긴 pandas DataFrame
        unigram일 경우 None
        bigram일 경우 unigram_df, trigram일 경우 bigram_df, ...        
    Returns
    ----------
    sentence_prob : float
        ngram language model을 사용하여 MLE를 계산하여 구한 확률
    '''
    if denominator_df is None:
      token_prob_denominator = numerator_df['count'].sum()
    sentence_prob = 1
    print('----\nToken\t\tProbability')
    for token in target_tokens:
        token_prob_numerator = numerator_df[numerator_df['token']==token]['count'].iloc[0]
        if denominator_df is not None: 
            token_prob_denominator = denominator_df[denominator_df['token']==token[:-1]]['count'].iloc[0]
        token_prob = token_prob_numerator / token_prob_denominator
        print('%s\t\t%.4f' % (' '.join(token), token_prob))
        sentence_prob *= token_prob
    return sentence_prob

print('Unigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 1), unigram_df))

----
Token		Probability
<s		0.0564
특히		0.0012
이에		0.0010
대한		0.0024
예산이		0.0001
충분히		0.0001
반영된다면		0.0000
좋은		0.0004
결과가		0.0001
있을		0.0004
것이라		0.0000
생각한다		0.0001
/s>		0.0564
Unigram Language Model에 의한 확률:  2.401683871402146e-45


## Bigram Language Model에 의한 확률
### Bigram Model (k=2): $P(w_i|w_1 w_2 ... w_{i-1}) \approx P(w_i|w_{i-1})$

\begin{equation}
    P(x_2|x_1) \approx \frac{count(x_1,x_2)}{count(x_1)}
\end{equation}

- P(*< s 특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다 /s >*)


In [11]:
bigram_df = ngram_df_using_defaultdict(flattened_sentences, n=2)

# bigram 빈도수를 기록한 dataframe 내 데이터 예시 출력
bigram_df.head()

,token,count
0,"(<s, 새로운)",49
1,"(새로운, 희망)",2
2,"(희망, 공유하고)",1
3,"(공유하고, 새)",2
4,"(새, 출발하자)",1


In [12]:
print('Bigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 2), bigram_df, unigram_df))

----
Token		Probability
<s 특히		0.0177
특히 이에		0.0005
이에 대한		0.0839
대한 예산이		0.0003
예산이 충분히		0.0104
충분히 반영된다면		0.0045
반영된다면 좋은		1.0000
좋은 결과가		0.0183
결과가 있을		0.0185
있을 것이라		0.0095
것이라 생각한다		0.2286
생각한다 /s>		0.4834
Bigram Language Model에 의한 확률:  3.098034360503449e-21


## Trigram Language Model에 의한 확률
\begin{equation}
    P(x_3|x_1,x_2) \approx \frac{count(x_1,x_2,x_3)}{count(x_1,x_2)}
\end{equation}

- P(*< s 특히 이에 대한 예산이 충분히 반영된다면 좋은 결과가 있을 것이라 생각한다 /s >*)

In [13]:
trigram_df = ngram_df_using_defaultdict(flattened_sentences, n=3)

# trigram 빈도수를 기록한 dataframe 내 데이터 예시 출력
trigram_df.head()

,token,count
0,"(<s, 새로운, 희망)",1
1,"(새로운, 희망, 공유하고)",1
2,"(희망, 공유하고, 새)",1
3,"(공유하고, 새, 출발하자)",1
4,"(새, 출발하자, /s>)",1


In [14]:
print('Trigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 3), trigram_df, bigram_df))

----
Token		Probability
<s 특히 이에		0.0006
특히 이에 대한		1.0000
이에 대한 예산이		0.0074
대한 예산이 충분히		1.0000
예산이 충분히 반영된다면		0.5000
충분히 반영된다면 좋은		1.0000
반영된다면 좋은 결과가		1.0000
좋은 결과가 있을		0.2500
결과가 있을 것이라		0.3333
있을 것이라 생각한다		0.6667
것이라 생각한다 /s>		0.7500
Trigram Language Model에 의한 확률:  9.307659086509106e-08


In [15]:
def print_elapsed_time(start):
    end = time.time()
    elapsed_time_txt = time.strftime("%H:%M:%S", time.gmtime(end - start))
    print('-----\nStart: %s, End: %s => Elapsed time: %s (%.4fs)' % (time.strftime("%H:%M:%S", time.gmtime(start)), time.strftime("%H:%M:%S", time.gmtime(end)), elapsed_time_txt, end-start), end='\n-----')

In [18]:
print('=====\n(위 코드를 통해 구현한) defaultdict 기반 ngram 추출 모듈')
start = time.time()
sentences = sentences_from_json(train_path)

flattened_sentences = ' '.join(sentences)
unigram_df = ngram_df_using_defaultdict(flattened_sentences, n=1)
bigram_df = ngram_df_using_defaultdict(flattened_sentences, n=2)
trigram_df = ngram_df_using_defaultdict(flattened_sentences, n=3)

print('문장 빈도에 의한 확률: ', sentence_probability_by_sentence_count(TARGET_SENT, sentences))
print('Unigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 1), unigram_df))
print('Bigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 2), bigram_df, unigram_df))
print('Trigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 3), trigram_df, bigram_df))

print_elapsed_time(start)

=====
(위 코드를 통해 구현한) defaultdict 기반 ngram 추출 모듈
문장 빈도에 의한 확률:  1.0703887651995204e-05
----
Token		Probability
<s		0.0564
특히		0.0012
이에		0.0010
대한		0.0024
예산이		0.0001
충분히		0.0001
반영된다면		0.0000
좋은		0.0004
결과가		0.0001
있을		0.0004
것이라		0.0000
생각한다		0.0001
/s>		0.0564
Unigram Language Model에 의한 확률:  2.401683871402146e-45
----
Token		Probability
<s 특히		0.0177
특히 이에		0.0005
이에 대한		0.0839
대한 예산이		0.0003
예산이 충분히		0.0104
충분히 반영된다면		0.0045
반영된다면 좋은		1.0000
좋은 결과가		0.0183
결과가 있을		0.0185
있을 것이라		0.0095
것이라 생각한다		0.2286
생각한다 /s>		0.4834
Bigram Language Model에 의한 확률:  3.098034360503449e-21
----
Token		Probability
<s 특히 이에		0.0006
특히 이에 대한		1.0000
이에 대한 예산이		0.0074
대한 예산이 충분히		1.0000
예산이 충분히 반영된다면		0.5000
충분히 반영된다면 좋은		1.0000
반영된다면 좋은 결과가		1.0000
좋은 결과가 있을		0.2500
결과가 있을 것이라		0.3333
있을 것이라 생각한다		0.6667
것이라 생각한다 /s>		0.7500
Trigram Language Model에 의한 확률:  9.307659086509106e-08
-----
Start: 06:23:56, End: 06:24:14 => Elapsed time: 00:00:17 (17.4100s)
-----

In [16]:
from collections import Counter
def ngram_df_using_counter(flattened_sentences, n):
    '''
    단어들로부터 ngram을 추출한다
    Parameters
    ----------
    flattened_sentences : string
        모든 문장들 사이에 빈칸을 두고 하나의 string으로 합친 것
    n : int
        unigram일 경우 1, bigram일 경우 2, trigram일 경우 3, ...
    Returns
    ----------
    df : pandas DataFrame
        모든 문장에 대해 추출한 ngram token 및 빈도수 (count) 값을 기록한 dataframe
    '''
    ngram_counter = Counter()   # defaultdict가 아닌 Counter를 통해 구현하며 바뀐 코드
    tokens = to_ngrams(flattened_sentences.split(), n)
    ngram_counter.update(tokens)   # defaultdict가 아닌 Counter를 통해 구현하며 바뀐 코드
    return pd.DataFrame(list(ngram_counter.items()), columns=['token', 'count'])

In [17]:
print('=====\n(위 코드를 통해 구현한) Counter 기반 ngram 추출 모듈')
start = time.time()
sentences = sentences_from_json(train_path)

flattened_sentences = ' '.join(sentences)
unigram_df = ngram_df_using_counter(flattened_sentences, n=1)
bigram_df = ngram_df_using_counter(flattened_sentences, n=2)
trigram_df = ngram_df_using_counter(flattened_sentences, n=3)

print('문장 빈도에 의한 확률: ', sentence_probability_by_sentence_count(TARGET_SENT, sentences))
print('Unigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 1), unigram_df))
print('Bigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 2), bigram_df, unigram_df))
print('Trigram Language Model에 의한 확률: ', sentence_prob_by_ngram_language_model(to_ngrams(TARGET_SENT.split(), 3), trigram_df, bigram_df))

print_elapsed_time(start)

=====
(위 코드를 통해 구현한) Counter 기반 ngram 추출 모듈
문장 빈도에 의한 확률:  1.0703887651995204e-05
----
Token		Probability
<s		0.0564
특히		0.0012
이에		0.0010
대한		0.0024
예산이		0.0001
충분히		0.0001
반영된다면		0.0000
좋은		0.0004
결과가		0.0001
있을		0.0004
것이라		0.0000
생각한다		0.0001
/s>		0.0564
Unigram Language Model에 의한 확률:  2.401683871402146e-45
----
Token		Probability
<s 특히		0.0177
특히 이에		0.0005
이에 대한		0.0839
대한 예산이		0.0003
예산이 충분히		0.0104
충분히 반영된다면		0.0045
반영된다면 좋은		1.0000
좋은 결과가		0.0183
결과가 있을		0.0185
있을 것이라		0.0095
것이라 생각한다		0.2286
생각한다 /s>		0.4834
Bigram Language Model에 의한 확률:  3.098034360503449e-21
----
Token		Probability
<s 특히 이에		0.0006
특히 이에 대한		1.0000
이에 대한 예산이		0.0074
대한 예산이 충분히		1.0000
예산이 충분히 반영된다면		0.5000
충분히 반영된다면 좋은		1.0000
반영된다면 좋은 결과가		1.0000
좋은 결과가 있을		0.2500
결과가 있을 것이라		0.3333
있을 것이라 생각한다		0.6667
것이라 생각한다 /s>		0.7500
Trigram Language Model에 의한 확률:  9.307659086509106e-08
-----
Start: 06:23:24, End: 06:23:39 => Elapsed time: 00:00:15 (15.2790s)
-----

In [ ]:
target_sent = "금메달을 획득해 전설로 남길 희망한다"